# Assignment 2

This assignment serves as a comprehensive evaluation of your machine learning skills, encompassing not only the technical aspects of model development but also your ability to analyze, interpret, and present data insights effectively. As such, it's essential to ensure that your submission is complete, functional, and devoid of any obvious gaps, as if you were delivering this project to a client.

To achieve this, leverage the full capabilities of Markdown and the interactive visualization tools available in Jupyter notebooks to craft a well-structured and visually appealing report of your findings. Your report should clearly communicate the insights you've gained from the exploratory data analysis, the rationale behind your data preprocessing and feature engineering decisions, and a thorough analysis of feature importance. High-quality visualizations and well-organized documentation will not only support your analysis but also make your results more accessible and understandable to your audience.

Remember, the ability to present complex results in an intuitive and engaging manner is a crucial skill, almost as important as the technical proficiency in model building and data analysis. Treat this assignment as an opportunity to showcase your skills in both areas.

## Instructions
- Your submission should be a `.ipynb` file with your name,
  like `FirstnameLastname.ipynb`. It should include the answers to the questions in markdown cells, your data analysis and results.
- You are expected to follow the best practices for code writing and model
training. Poor coding style will be penalized.
- You are allowed to discuss ideas with your peers, but no sharing of code.
Plagiarism in the code will result in failing. If you use code from the
internet, cite it by adding the source of the code as a comment in the first line of the code cell. [Academic misconduct policy](https://wiki.innopolis.university/display/DOE/Academic+misconduct+policy)
- In real life clients can give unclear goals or requirements. So, if the instructions seem vague, use common sense to make reasonable assumptions and decisions.

## Self-Reliance and Exploration
In this task, you're encouraged to rely on your resourcefulness and creativity. Dive into available resources, experiment with various solutions, and learn from every outcome. While our team is here to clarify task details and offer conceptual guidance, we encourage you to first seek answers independently. This approach is vital for developing your problem-solving skills in machine learning.



# Task 2: Image Classification with CNNs (50%)

In this task, you'll dive into the world of Convolutional Neural Networks (CNNs) by working with the CIFAR-10 dataset, a staple in image classification challenges. Your goal is to build and evaluate two different CNN models to classify images into one of the ten categories accurately.

The dataset is availabel in pytorch and keras.

## Part 1: Custom CNN Model (20%)

- Design and train a CNN model from scratch tailored for the CIFAR-10 dataset.
- Focus on the architecture that you believe will perform best for this specific task.
- Integrate various techniques such as batch normalization, dropout, learning rate schedulers, and early stopping to improve model training. Experiment with these methods and finetune them to see how they affect training stability, convergence speed, and overall performance.

## Part 2: Transfer Learning Model (20%)

- Implement a transfer learning approach using a pre-trained model of your choice.
- Fine-tune the model on the CIFAR-10 dataset to achieve the best possible performance.

## Evaluation (10%)

Ensure that both models are robust and generalized well to unseen data.

After training both models, you will evaluate them on a provided test dataset.

Compare your models based on:
- **AUC-ROC**: How well does each model discriminate between classes?
- **Model Size**: Consider the trade-offs in model complexity.
- **Inference Speed**: Evaluate how quickly your model can predict classes for new images.

Reflect on the performance, size, and inference speed of both models. What insights can you draw from these comparisons?

### Learning Objectives

- Understand and apply CNNs for image classification.
- Explore the impact of model architecture on performance and efficiency.
- Learn the process and benefits of transfer learning in deep learning.

Remember, the key to this task is not just about achieving the highest accuracy but also understanding the strengths and limitations of different approaches in machine learning model development.

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils import data
from torchvision import datasets, transforms

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 1: Load CIFAR-10 dataset
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# testloader = DataLoader(testset, batch_size=64, shuffle=False)
train_batch_size = 128
test_batch_size = 128
train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=2),
    transforms.RandomHorizontalFlip(), # FLips the image w.r.t horizontal axis
    transforms.RandomRotation(10),     #Rotates the image to a specified angel
    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
# Do not modify test transforms, because it will corrupt test data
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


# You should get how to get the training part of datasets.CIFAR10 and
# how to apply train_transforms here
# Specify path of downloaded set in root, if you've loaded it
train_dataset = datasets.CIFAR10(root='cifar10',
                                 train=True,
                                 transform=train_transforms,
                                 download=True)

# Define the loader of train data set: use train_batch_size as batch_size
# and don't forget to shuffle
trainloader = data.DataLoader(train_dataset,
                                    batch_size=train_batch_size,
                                    shuffle=True,
                                    drop_last=True,
                                    num_workers=2)


# Do the same for test part of CIFAR10, apply test_transforms
test_dataset = datasets.CIFAR10(root='cifar10',
                                 train=False,
                                 transform=test_transforms,
                                 download=True)

# Define the loader of test data set: use test_batch_size
testloader = data.DataLoader(test_dataset,
                                    batch_size=test_batch_size,
                                    shuffle=False,
                                    num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [26]:

# Step 2: Define the CNN architecture
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.batchnorm3 = nn.BatchNorm2d(128)

    def forward(self, x):
        x = self.relu(self.batchnorm1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.batchnorm2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.batchnorm3(self.conv3(x)))
        x = self.pool(x)
        x = x.view(-1, 128 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Step 3: Configure training parameters
model = CustomCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)
early_stopping = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

# Step 4: Train the model
epochs = 50
best_val_loss = float('inf')
patience = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(trainloader.dataset)
    train_accuracy = 100 * correct / total
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(testloader.dataset)
    val_accuracy = 100 * correct / total
    
    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience = 0
    else:
        patience += 1
        if patience > 5:
            print("Early stopping...")
            break
    
    scheduler.step(val_loss)

# Step 5: Evaluate the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy:.2f}%')

Epoch 1/50, Train Loss: 1.5834, Train Accuracy: 41.85%, Validation Loss: 1.2443, Validation Accuracy: 54.77%
Epoch 2/50, Train Loss: 1.2888, Train Accuracy: 53.37%, Validation Loss: 1.0210, Validation Accuracy: 63.37%
Epoch 3/50, Train Loss: 1.1519, Train Accuracy: 58.85%, Validation Loss: 0.8984, Validation Accuracy: 67.71%
Epoch 4/50, Train Loss: 1.0717, Train Accuracy: 62.04%, Validation Loss: 0.9102, Validation Accuracy: 68.08%
Epoch 5/50, Train Loss: 1.0162, Train Accuracy: 64.18%, Validation Loss: 0.8486, Validation Accuracy: 69.47%
Epoch 6/50, Train Loss: 0.9710, Train Accuracy: 65.69%, Validation Loss: 0.8519, Validation Accuracy: 70.90%
Epoch 7/50, Train Loss: 0.9310, Train Accuracy: 67.59%, Validation Loss: 0.7422, Validation Accuracy: 73.48%
Epoch 8/50, Train Loss: 0.8969, Train Accuracy: 68.41%, Validation Loss: 0.7003, Validation Accuracy: 75.15%
Epoch 9/50, Train Loss: 0.8770, Train Accuracy: 69.67%, Validation Loss: 0.7383, Validation Accuracy: 74.05%
Epoch 10/50, Train 

In [27]:
%pip install pygame
import pygame

def play_sound(file_path):
    pygame.init()
    pygame.mixer.init()
    
    try:
        sound = pygame.mixer.Sound(file_path)
        sound.play()
        pygame.time.wait(int(sound.get_length() * 1000))  # Ждем, пока звук проиграется
    except pygame.error as e:
        print(f"Error playing sound: {e}")

# Пример использования
sound_file_path = "sound.mp3"
play_sound(sound_file_path)

Note: you may need to restart the kernel to use updated packages.


In [29]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision
# import torchvision.transforms as transforms
# from torch.utils.data import DataLoader

# # Check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # # Step 1: Load CIFAR-10 dataset
# # transform = transforms.Compose([
# #     transforms.ToTensor(),
# #     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# # ])

# # trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# # trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

# # testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# # testloader = DataLoader(testset, batch_size=64, shuffle=False)

# # # Step 2: Load a pre-trained model
# pretrained_model = torchvision.models.resnet50(pretrained=True)
# for param in pretrained_model.parameters():
#     param.requires_grad = False

# pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, 10)  # Change the output layer to match CIFAR-10

# # # Freeze all the layers except the last one
# # for param in pretrained_model.parameters():
# #     param.requires_grad = False
# pretrained_model.fc.requires_grad = True

# # Move the model to the device
# pretrained_model = pretrained_model.to(device)

# # Step 3: Configure training parameters
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(pretrained_model.parameters(), lr=0.01)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=20, verbose=True)
# early_stopping = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=20, verbose=True)

# # Step 4: Train the model
# epochs = 20
# best_val_loss = float('inf')
# patience = 0

# for epoch in range(epochs):
#     pretrained_model.train()
#     train_loss = 0.0
    
#     for inputs, labels in trainloader:
#         inputs, labels = inputs.to(device), labels.to(device)
        
#         optimizer.zero_grad()
#         outputs = pretrained_model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         train_loss += loss.item() * inputs.size(0)
    
#     train_loss = train_loss / len(trainloader.dataset)
    
#     # Validation
#     pretrained_model.eval()
#     val_loss = 0.0
    
#     with torch.no_grad():
#         for inputs, labels in testloader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = pretrained_model(inputs)
#             loss = criterion(outputs, labels)
#             val_loss += loss.item() * inputs.size(0)
    
#     val_loss = val_loss / len(testloader.dataset)
    
#     print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
    
#     # Early stopping
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         patience = 0
#     else:
#         patience += 1
#         if patience > 5:
#             print("Early stopping...")
#             # break
    
#     scheduler.step(val_loss)

# # Step 5: Evaluate the model
# pretrained_model.eval()
# correct = 0
# total = 0

# with torch.no_grad():
#     for inputs, labels in testloader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         outputs = pretrained_model(inputs)
#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# accuracy = 100 * correct / total
# print(f'Accuracy on test set: {accuracy:.2f}%')


KeyboardInterrupt: 

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Check if GPU is available

# Step 1: Load CIFAR-10 dataset with data augmentation

# Step 2: Load a pre-trained model
pretrained_model = torchvision.models.resnet50(pretrained=True)

# for param in pretrained_model.parameters():
#     param.requires_grad = False


pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, 10)
# pretrained_model.fc.requires_grad = True
  # Change the output layer to match CIFAR-10
pretrained_model = pretrained_model.to(device)

# Step 3: Configure training parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pretrained_model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, verbose=True)

# Step 4: Train the model
epochs = 30
best_val_loss = float('inf')
patience = 0
best_accuracy = 0

for epoch in range(epochs):
    pretrained_model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = pretrained_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = train_loss / len(trainloader.dataset)
    train_accuracy = 100 * correct / total
    
    # Validation
    pretrained_model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = pretrained_model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss = val_loss / len(testloader.dataset)
    val_accuracy = 100 * correct / total
    
    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')
    if best_accuracy < val_accuracy:
        best_accuracy = val_accuracy
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience = 0
    else:
        patience += 1
        if patience > 5:
            print("Early stopping...")
            break
    
    scheduler.step(val_loss)

# Step 5: Evaluate the model
pretrained_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = pretrained_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {best_accuracy:.2f}%')


Epoch 1/30, Train Loss: 1.1054, Train Accuracy: 61.31%, Validation Loss: 0.7097, Validation Accuracy: 75.47%
Epoch 2/30, Train Loss: 0.7552, Train Accuracy: 73.72%, Validation Loss: 0.5835, Validation Accuracy: 79.76%


KeyboardInterrupt: 